## Setup Drive and Data Folder

In [ ]:
import pandas as pd
import os

PRECISION = 4 # 3 of digits to keep after the decimal point

RUNNING_ON_COLAB = True # we assume running on CoLab! Change to False if running locally.

*italicized text*

In [ ]:
## mount gdrive
! pip install google.colab
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
# If running locally, define current working path
path = os.getcwd()

# If Google colab
if RUNNING_ON_COLAB:
  path = "/content/gdrive/My Drive/Colab Notebooks/RandomForest_FeatureData"

print(path)

# define current data path. This is after we did classification. We have done some
# cleaning already.
data_path = path + '/results'

## Util - add RUL column

In [ ]:
csv_file = data_path + '/combined_offset_misalignment.csv'
df_temp = pd.read_csv(csv_file, chunksize=50000) 
df = pd.concat(df_temp, ignore_index=True)

In [ ]:
# Let's find the youngest & oldest timestamp

df['wf_start_time'] = pd.to_datetime(df['wf_start_time']) # make sure it is datetime

youngest = min(df.wf_start_time)
oldest = max(df.wf_start_time)
print(youngest)
print(oldest)
span = oldest - youngest
print(span)
print(span.total_seconds())

## Using Oldest - current to determine the RUL
df['rul'] = df['wf_start_time'].apply(lambda x: (oldest - x).total_seconds())

In [ ]:
# save back with RUL
df.to_csv(path + '/results/combined_offset_misalignment_with_RUL.csv')


## LSTM Regression

In [ ]:
## Common imports
colab = True

import pandas as pd
import numpy as np
import csv
import os

from sklearn import tree
from sklearn import metrics
from sklearn.metrics import f1_score, cohen_kappa_score

from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Input
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping


from sklearn.metrics import r2_score, mean_squared_error
# from pandas_ml import ConfusionMatrix

import matplotlib
import matplotlib.pyplot as plt
plt.style.available
%matplotlib inline

# to make this notebook's output stable across runs
RNDSEED = np.random.seed(39)

### Exploring the data a bit & prep the df

In [ ]:
## future, let's read the file
if colab == False:
  csv_file = "C:\\Users\\colte\\Downloads\\combined_offset_misalignment_with_RUL.csv"
else:
  csv_file = path + '/results/combined_offset_misalignment_with_RUL.csv'
df_temp = pd.read_csv(csv_file, chunksize=50000) 
big_df = pd.concat(df_temp)

In [ ]:
df = big_df # reset 

# drop unwanted cols
df = df[df.columns.drop(list(df.filter(regex='Unnamed')))] # drop Unnamed
df = df[df.columns.drop(list(df.filter(regex='wf_start_time')))] # drop time column
df = df[df.columns.drop(list(df.filter(regex='status')))] # drop status column



In [ ]:
df.shape

In [ ]:
df.columns

### RF works

In [ ]:
# Get X & y
# Naming convention: X as predictors; y as response.
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
import numpy as np


# Set the sequence length and batch size
seq_length = 100
batch_size = 32


scaler = MinMaxScaler()

y = df['rul'] # pop response
unscaled_X = df.drop('rul',axis = 1) # drop response
X = scaler.fit_transform(unscaled_X)
X_scaled_with_y = pd.concat([pd.DataFrame(X, columns=unscaled_X.columns), y], axis=1)

print(X)

# Create the TimeseriesGenerator object
data_generator = TimeseriesGenerator(data=X_scaled_with_y.drop('rul', axis=1).values,
                                      targets=X_scaled_with_y['rul'].values,
                                      length=seq_length,
                                      batch_size=batch_size)



# print(len(df))
# Extract the data and targets from the generator
x_train = []
y_train = []
for i in range(len(data_generator)):
    x, y = data_generator[i]
    x_train.append(x)
    y_train.append(y)
X_train_seq = np.concatenate(x_train)
y_train_seq = np.concatenate(y_train)

# Print the shape of the input and target data
print('Input data shape:', X_train_seq.shape)
print('Target data shape:', y_train_seq.shape)





print (X.shape)
# print (X.columns)

In [ ]:
## true orignal ones


# data split
X_train, X_test, y_train, y_test = train_test_split(X_train_seq, y_train_seq, random_state=RNDSEED)

print(y_train.shape[0], y_test.shape[0])

In [ ]:
y_test.shape

In [ ]:
len(X_train.dtypes)


In [ ]:
y_train.dtypes


In [ ]:
from sklearn.metrics import mean_squared_error
from keras.regularizers import l2


# n_steps = seq_length
# n_features = 46
# n_samples = len(X_test)

# X_test1 = X_test.to_numpy()

# X_test2 = X_test.reshape(n_samples, n_steps, n_features)

# X_train1 = X_train.to_numpy()
# print(X_train)

def LSTM_RUL():
  n_steps = seq_length
  n_features = 46
  n_samples = len(X_train)

  model = Sequential()
  model.add(LSTM(256, activation='tanh', input_shape = (n_steps, n_features)))
  model.add(Dense(units=1, activation='relu'))
  # model.add(LSTM(64, activation='relu', input_shape = (n_steps, n_features)))

  # model.add(Dense(units=1, activation='sigmoid'))
  model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))

  # print(len(X_train1))

  # X_train2 = X_train.reshape(n_samples, n_steps, n_features)
  # print(X_train2)

  # fit model

  early_stopping = EarlyStopping(monitor='loss', patience=5, verbose=1, mode='min')

  history = model.fit(X_train, y_train, epochs=50, callbacks=[early_stopping])

  loss_history = history.history['loss']

  # Plot the loss history
  plt.plot(np.arange(len(loss_history)), loss_history, label='Training Loss')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.legend()
  plt.show()
  return model

L = LSTM_RUL()



# Predict on the test data
y_pred = L.predict(X_test)

# Calculate R-squared
r2 = r2_score(y_test, y_pred)
print("R2 score:", r2)

loss = L.evaluate(X_test, y_test)
print("Test loss:", loss)


### Save/Load the model

In [ ]:
## utility - save/load the model
import os
import joblib

# to save
# joblib.dump(rf, path + '/results/random_forest_offset_RUL.joblib') 

# to load
rf = joblib.load(path + '/results/random_forest_offset_RUL.joblib')

### Evaluation

In [ ]:
y_pred = rf.predict(X_test) ## using the untinted dataset!
    
print('R^2:', metrics.r2_score(y_test, y_pred))
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, y_pred))
print('Mean Absolute Percentage Error (MAPE):', metrics.mean_absolute_percentage_error(y_test, y_pred))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, y_pred))) # np.sqrt

print('Explained Variance Score:', metrics.explained_variance_score(y_test, y_pred))
print('Max Error:', metrics.max_error(y_test, y_pred))
print('Mean Squared Log Error:', metrics.mean_squared_log_error(y_test, y_pred))
print('Median Absolute Error:', metrics.median_absolute_error(y_test, y_pred))

## with n_estimators = 150

# R^2: 0.9991654290573937
# Mean Absolute Error (MAE): 1158.6216816072117
# Mean Squared Error (MSE): 333818221.94037396
# Root Mean Squared Error (RMSE): 18270.692979205083
# Explained Variance Score: 0.9991654316522736
# Max Error: 1998236.0866666667
# Mean Squared Log Error: 0.012717975468372306
# Median Absolute Error: 147.4266666667536

## n_estimators = 10
# R^2: 0.9990822069686238
# Mean Absolute Error (MAE): 1213.5715068447246
# Mean Squared Error (MSE): 367106044.79767376
# Mean Absolute Percentage Error (MAPE): 1.0690696275079867e+17
# Root Mean Squared Error (RMSE): 19160.011607451437
# Explained Variance Score: 0.9990822141398445
# Max Error: 1998818.5
# Mean Squared Log Error: 0.012296657672986336
# Median Absolute Error: 137.60000000009313

### Show feature importance

In [ ]:
# RF: Get feature list
## Learn more: https://towardsdatascience.com/explaining-feature-importance-by-example-of-a-random-forest-d9166011959e

features = list(X_test.columns)  

rf_imp_features = []

## Plot the feature importance
def plot_feature_importance ():
    importances = rf.feature_importances_
    
    indices = np.argsort(importances)[len(importances)-25:] ## top 25    

    plt.figure(figsize = (12,8))
    plt.rcParams['font.size'] = 12
    
    plt.title('Feature Importances')
    plt.barh(range(len(indices)), importances[indices], color='b', align='center')

    features_y = []
    for x in indices:
        features_y.append(features[x])
        
    plt.yticks(range(len(indices)), features_y) 

    plt.xlabel('Relative Importance')
    plt.savefig(path + '/results/fi_offset_RUL.png') # save

    plt.show()
    return features_y

rf_imp_features = plot_feature_importance()

### Prediction

In [ ]:
# get a random sample to verify the results!
## 
import matplotlib
import matplotlib.pyplot as plt
RNDSEED = np.random.seed(39)

dfs = df.sample(1000, random_state = RNDSEED) # data points
# print(dfs.rul)
X_dfs = dfs.drop('rul',axis = 1) # drop response

ys_pred = rf.predict(X_dfs)
# print(ys_pred)

rul = dfs[['rul']]
rul = rul.rename(columns={'rul': 'original'})
rul['prediction'] = ys_pred
# rul.head(5)

matplotlib.style.use('ggplot') ## styling

rul.plot.scatter(x='original', y='prediction', figsize=(16, 9), c='original', colormap='viridis') ## scatter

plt.savefig(path + '/results/offset_rul_prediction.png') # save


In [ ]:
## Plot the % of deviation 

rul['difference'] = (rul['prediction'] - rul['original']) /  rul['original']

rul.index = range(len(rul.index)) ## reset index

ax = rul.plot( y=["difference"], figsize=(16, 9))

ax.set_xlabel("Index")
ax.set_ylabel("Difference")

plt.show()

# out of couriosity 
print(len(rul[(rul['difference'] > 0.2) | (rul['difference'] < -0.2)])) # how many more than 20%? # 8 only!

In [ ]:
## styles
plt.style.available
